In [1]:
import pandas as pd
import json
import math
import datetime

In [2]:
def str_split (url):
    
    if isinstance(url, str):
        urlst = url.split(', ')
        return urlst
    else: # no url
        return None

def time_split(s):
    if isinstance(s, str):
        slst = s.split(' | ')
        return slst
    else: # no url
        return None

def str_to_dict (s):
    return(json.loads(s))

ramenRest and ramenReview

In [3]:
all_rest = []
linelst = ['red', 'yellow', 'green', 'blue', 'orange', 'brown', 'airport']
info_col = ['name', 'site', 'phone', 'full_address', 'mrt', 'rating', 'reviews', 'reviews_link', 'reviews_tags', 'reviews_per_score', 'photo', 'working_hours_old_format', 'description', 'location_link', 'place_id', 'walk_distance', 'walk_time', 'food_regular', 'food_limited']
review_col = ['author_title', 'review_text', 'review_img_urls', 'review_link', 'review_rating', 'review_datetime_utc']
ig_col = ['ig_writer', 'content', 'time', 'url']
rest_id = 0

for line in linelst:

    infodf = pd.read_excel('./ramenRest.xlsx', line) # rest info
    rvdf = pd.read_excel('./ramenReview.xlsx', line) # rest reviews
    igdf = pd.read_excel('./ig_review.xlsx') # ig reviews

    # For each restaurant
    for i in range(len(infodf)):
        rest_dict = {}
        rest_dict['line'] = line
        rest_dict['id'] = rest_id
        rest_id += 1

        for j in range(len(info_col)): 
            dkey = info_col[j]

            if dkey == 'reviews': # Object of type int64 is not JSON serializable
                rest_dict[dkey] = int(infodf.iloc[i][info_col[j]]) 
            elif dkey == 'reviews_tags' or dkey == 'food_regular' or dkey == 'food_limited': # split by ', '
                rest_dict[dkey] = str_split(infodf.iloc[i][info_col[j]]) 
            elif dkey == 'reviews_per_score': # str to dict
                rest_dict[dkey] = str_to_dict(infodf.iloc[i][info_col[j]])
            elif dkey == 'working_hours_old_format' :
                rest_dict[dkey] = time_split(infodf.iloc[i][info_col[j]])
            else:
                rest_dict[dkey] = infodf.iloc[i][info_col[j]]
            

        rvdf2 = rvdf[rvdf.name == rest_dict['name']] # filter reviews by rest's name
        review_list = []
        
        for k in range(len(rvdf2)): # for a rest's each review 
            rv_dict = {}

            for c in range(len(review_col)):
                dkey2 = review_col[c]

                if dkey2 == 'review_img_urls':
                    url = rvdf2.iloc[k][review_col[c]]
                    rv_dict[dkey2] = str_split(url)
                elif dkey2 == 'review_rating': # Object of type int64 is not JSON serializable
                    rv_dict[dkey2] = int(rvdf2.iloc[k][review_col[c]])
                else:
                    rv_dict[dkey2] = rvdf2.iloc[k][review_col[c]]

            review_list.append(rv_dict)
        
        rest_dict['review_list'] = review_list

        igdf2 = igdf[igdf.name == rest_dict['name']]
        ig_list = []

        for m in range(len(igdf2)):
            ig_dict = {}

            for n in range(len(ig_col)):
                dkey3 = ig_col[n]
                ig_dict[dkey3] = igdf2.iloc[m][ig_col[n]]
            
            ig_list.append(ig_dict)
        
        rest_dict['ig_list'] = ig_list


        all_rest.append(rest_dict) # add a rest to list
       
    print(f'{line}: {len(all_rest)} rests')
    # all_rest[line] = restlst # add list to corresponding line
    

red: 63 rests
yellow: 76 rests
green: 103 rests
blue: 141 rests
orange: 174 rests
brown: 209 rests
airport: 217 rests


In [4]:
all_rest[0]

{'line': 'red',
 'id': 0,
 'name': '鷄道樂',
 'site': 'https://www.facebook.com/%E9%B7%84%E9%81%93%E6%A8%82-105773151987737',
 'phone': '+886 2 2625 1088',
 'full_address': '251新北市淡水區公明街55-2號',
 'mrt': '淡水',
 'rating': 4.3,
 'reviews': 348,
 'reviews_link': 'https://search.google.com/local/reviews?placeid=ChIJucLSih-lQjQR1NpiypNOwTo&q=%E9%9B%9E%E9%81%93%E6%A8%82&authuser=0&hl=zh&gl=TW',
 'reviews_tags': ['雞湯', '雞白湯', '松露', '細麵', '溫和', '傳統'],
 'reviews_per_score': {'1': 19, '2': 10, '3': 27, '4': 67, '5': 225},
 'photo': 'https://lh5.googleusercontent.com/p/AF1QipOFVP5fPQeqR7uu58KZDNv4Pd9JSjVIv0zH96Yh=w800-h500-k-no',
 'working_hours_old_format': ['星期一: 12:00-14:00,17:00-21:00',
  '星期二: 12:00-14:00,17:00-21:00',
  '星期三: 12:00-14:00,17:00-21:00',
  '星期四: 12:00-14:00,17:00-21:00',
  '星期五: 12:00-14:00,17:00-21:00',
  '星期六: 12:00-14:00,17:00-21:00',
  '星期日: 12:00-14:00,17:00-21:00'],
 'description': '設有吧台座位的閒適小餐館，供應經典拉麵。',
 'location_link': 'https://www.google.com/maps/place/%E9%B7%84%E9%81%93

Restaurant

In [5]:
Restaurant = []
daylst = ['Mon', 'Tue', 'Wed', 'Thr', 'Fri', 'Sat', 'Sun']

for rest in all_rest:
    info = {}

    info['id'] = rest['id']
    info['name'] = rest['name']
    info['site'] = rest['site']
    info['phone'] = rest['phone']
    info['address'] = rest['full_address']
    info['line'] = rest['line']
    info['mrt'] = rest['mrt']
    info['rating'] = rest['rating']
    info['tag'] = rest['reviews_tags']
    info['img'] = rest['photo']

    time_dict = {}
    if rest['working_hours_old_format'] != None:
        for day in range(len(rest['working_hours_old_format'])):
            time_dict[daylst[day]] = rest['working_hours_old_format'][day][5:]
        
    info['time'] = time_dict
    
    info['distance'] = rest['walk_distance']
    info['walktime'] = rest['walk_time']
    info['regular'] = rest['food_regular']
    info['limited'] = rest['food_limited']

    Restaurant.append(info)

In [6]:
type(all_rest[8]['working_hours_old_format'])

NoneType

In [7]:
Restaurant[0]

{'id': 0,
 'name': '鷄道樂',
 'site': 'https://www.facebook.com/%E9%B7%84%E9%81%93%E6%A8%82-105773151987737',
 'phone': '+886 2 2625 1088',
 'address': '251新北市淡水區公明街55-2號',
 'line': 'red',
 'mrt': '淡水',
 'rating': 4.3,
 'tag': ['雞湯', '雞白湯', '松露', '細麵', '溫和', '傳統'],
 'img': 'https://lh5.googleusercontent.com/p/AF1QipOFVP5fPQeqR7uu58KZDNv4Pd9JSjVIv0zH96Yh=w800-h500-k-no',
 'time': {'Mon': '12:00-14:00,17:00-21:00',
  'Tue': '12:00-14:00,17:00-21:00',
  'Wed': '12:00-14:00,17:00-21:00',
  'Thr': '12:00-14:00,17:00-21:00',
  'Fri': '12:00-14:00,17:00-21:00',
  'Sat': '12:00-14:00,17:00-21:00',
  'Sun': '12:00-14:00,17:00-21:00'},
 'distance': '1.8 km',
 'walktime': '16 mins',
 'regular': ['豪華雞清湯拉麵', '雞白湯拉麵'],
 'limited': None}

Comment

In [8]:
Comment = []

for rest in all_rest:
    if len(rest['review_list']) > 0:
        for i in range(len(rest['review_list'])):
            comm = {}
            comm['restaurantId'] = rest['id']
            comm['author'] = rest['review_list'][i]['author_title']
            comm['text'] = rest['review_list'][i]['review_text']

            if rest['review_list'][i]['review_img_urls'] != None:
                comm['imgurl'] = rest['review_list'][i]['review_img_urls'][0]
            else:
                comm['imgurl'] = None

            comm['rating'] = rest['review_list'][i]['review_rating']
            
            Comment.append(comm)


In [9]:
Comment

[{'restaurantId': 0,
  'author': 'Grace Tu',
  'text': '雞湯真的有濃有讚，舒肥雞也有嫩，加點的蔥如果是蔥花會更喜，愛雞白湯的一定愛',
  'imgurl': 'https://lh5.googleusercontent.com/p/AF1QipM3w3gb62DpP1VxTjz2u10wExtRm7F5UuY-PLhn',
  'rating': 5},
 {'restaurantId': 0,
  'author': 'Chuang Mint',
  'text': '來那麼多次了，第一次給一顆星，沒點叉燒飯給叉燒飯，跟店員說還說我們有點，其實我可以直接端下來吃掉的，店員發現送錯也沒有感到抱歉，只說，沒事！然後端給隔壁的客人，加點的叉燒肉死活就是不給盤子或是碗裝，堅持要放在點的三碗麵裡的其中一碗或是一碗放一片，態度差！水杯一倒水就有油垢浮出！環境清潔有待加強，排隊時的規則，店員也不會確時執行！排隊數人數，我前面明明一個人，報2個人也是讓她排隊，我們眼睜睜看著她同行的友人直接解壓縮，排隊規則可以直接刪掉第一條了！',
  'imgurl': None,
  'rating': 1},
 {'restaurantId': 0,
  'author': '單耘甜',
  'text': '店員默認排隊解壓縮，排隊的時候看到不只一組只有一個人排，進去的卻有2、3人，明明店門口排隊公告就明文規定不可代排解壓縮，讓人看了覺得莫名其妙。',
  'imgurl': None,
  'rating': 1},
 {'restaurantId': 0,
  'author': '高源治',
  'text': '非常好吃的拉麵，價格不貴，可以免費加一碗麵，店面座位較少，晚上常常要排隊30分鐘-1小時不等\n\n——————————————————-\n更新：價格有稍微調漲',
  'imgurl': 'https://lh5.googleusercontent.com/p/AF1QipOZeSQITuL2DaZMT28Sas2Y3CU6rKfc9kqQ7jx4',
  'rating': 4},
 {'restaurantId': 0,
  'author': '華黎林',
  'text': '豪華清鷄湯拉麵～\n麵是細麵

Post

In [10]:
Post = []

for rest in all_rest:
    if len(rest['ig_list']) > 0:
        for i in range(len(rest['ig_list'])):
            p = {}
            p['restaurantId'] = rest['id']
            p['writer'] = rest['ig_list'][i]['ig_writer']
            p['content'] = rest['ig_list'][i]['content']
            p['time'] = rest['ig_list'][i]['time']
            p['url'] = rest['ig_list'][i]['url']
            Post.append(p)

In [11]:
Post

[{'restaurantId': 0,
  'writer': '拉麵浪人｜拉麵食記 美食資訊㊙️有趣分享',
  'content': '「咬勁俐落的麵體搭配細緻多芳的湯頭，吃來輕巧但又內斂沉穩，平淡中讓人深刻。」\n\u2061\n▎Bowl 40\n▎店名：#雞道樂\n▎地點：淡水捷運站\n▎排隊狀況：18:40到前方6人，等半小時入場\n▎店內座位：約13個 \n▎拉麵名稱：#豪華雞清湯拉麵\n▎配置：舒肥豬叉燒 / 舒肥雞叉燒 / 溏心蛋 / 海老餛飩／洋蔥 \n▎價格：250 + 30 (海帶芽) \n▎喜好度：4. 25 / 5\n▎MVP：麵條\n\u2061\n\u2061\n二訪雞道樂，發現和上次比起來，菜單的定番多了一碗干貝雞清湯拉麵，不過已售完…\n在排隊時仔細研究菜單，發現一件有趣的事情，就是這邊的雞清湯反而比雞白湯貴，好像和一般的認知不太一樣，今天天氣剛好有點濕黏，所以不想吃太厚重的，所以就選了這碗豪華雞清湯。\n\u2061\nPs. 在我點餐時叉燒飯顯示售完，但是進去沒多久就看到店員再把選項打開，可能是原本白飯用完了，剛好在我點完之後煮好，所以如果很想吃叉燒飯但又剛好看到售完的話，建議可以問一下店員。\n\u2061\n\u2061\n湯頭 ｜喜好度 4.5\n以塩味雞湯為底，吃起來鹹度恰到好處，不會讓人覺得淡而無味，也不會鹹到讓人頻頻喝水，淡淡的雞香味喝起來十分甘甜清爽。有些雞清湯會用比較多的雞油來加強風味層次，這碗卻沒有那種油膩的感覺，同時又不讓人覺得太過寡薄，湯質厚而不膩，搭配清甜，且沒有嗆辣味的碎洋蔥末稍稍轉味，吃起來相當舒服。\n\u2061\n\u2061\n麵條 ｜喜好度 4\n細直麵，我選擇麵硬，不是那種太硬脆的齒切感，麵條吸水之後口感更為彈潤，不會軟爛，吃起來相當精神抖擻，且帶湯力也很不錯，咬勁俐落的麵體與細緻多芳的湯頭，搭配起來頗為契合，都是吃來輕巧但又內斂沉穩讓人深刻的類型。\n\u2061\n\u2061\n配料｜喜好度 4\n舒肥豬叉燒，味道是傳統的那種淡淡豬香味，最近吃到比較多類培根的燻香豬叉燒，覺得有點膩，再回頭來吃這樣比較樸實的味道反而覺得很喜歡，而且肉質軟嫩，厚度也剛好，咬起來相當輕鬆且不時溢出肉汁，不強勢的風味給人一種漸入佳境的感覺。雞叉燒也是類似的情況，沒有太過重的調味，以雞肉自己的味道搭配湯頭

In [38]:
with open('./restData.json', 'w') as f:
    json.dump(all_rest, f, ensure_ascii = False)

In [76]:
with open('./Restaurant.json', 'w') as f:
    json.dump(Restaurant, f, ensure_ascii = False)

In [77]:
with open('./Comment.json', 'w') as f:
    json.dump(Comment, f, ensure_ascii = False)

In [78]:
with open('./Post.json', 'w') as f:
    json.dump(Post, f, ensure_ascii = False)